In [ ]:
import os
import numpy
import pandas
import re

from capblood_seq import config

from scrapi.dataset import Gene_Expression_Dataset as GED

In [ ]:
STUDIES = [
    "Hashimoto2019",
    "Hu2019",
    "Lee2020"
]

In [ ]:
cell_type_labels = []

for cell_type in config.CELL_TYPES:
    
    if cell_type in config.CELL_SUBTYPES:

        for cell_subtype in config.CELL_SUBTYPES[cell_type]:

            cell_type_labels.append("%s %s" % (cell_subtype, cell_type))
    else:
        cell_type_labels.append(cell_type)

for study in STUDIES:
        
    df = pandas.DataFrame(columns=cell_type_labels)
    
    labels_file_path = os.path.join("data", study, "labels.csv")
    _, label_barcodes = GED.get_label_cells_from_file(labels_file_path)
    
    subject_barcodes = {}
    cell_type_barcodes = {}
    
    for label, cell_barcodes in label_barcodes.items():
        
        if re.match("S[0-9]+", label):
            subject_barcodes[label] = cell_barcodes
        else:
            cell_type_barcodes[label] = cell_barcodes
            
    for subject_id, cell_barcodes in subject_barcodes.items():

        series = pandas.Series(index=cell_type_labels, dtype=numpy.float32)
        
        for cell_type in cell_type_labels:
            series[cell_type] = len(set(cell_barcodes).intersection(cell_type_barcodes[cell_type]))
        
        df.loc["%s" % (subject_id)] = series
        
    print(study)
#     display(df)
    df_percentiles = df.divide(df.sum(axis=1), axis=0)*100
    display(df_percentiles)
    df_percentiles.to_csv(os.path.join("data", "%s_cell_type_percentages.csv" % study))